In [13]:
import numpy as np
from typing import List, Tuple, Dict, Union, Optional, NewType
from __future__ import annotations

In [2]:
npmatrix = NewType('npmatrix', np.matrix)
nparray = NewType('nparray', np.ndarray)

# 0. Tool functions

In [3]:
from functools import reduce

# eg. Dag(|a>) = <a|
Dag = lambda matrix: matrix.conj().T
# eg. Kron(I, X, Y) = I ⊗ X ⊗ Y，计算张量用
Kron = lambda *matrices: reduce(np.kron, matrices)

In [4]:
I = np.eye(2)

# pauli matrixes
X = np.matrix([
    [0, 1], [1, 0]
])
Y = np.matrix([
    [0, -1j], [1j, 0]
])
Z = np.matrix([
    [1, 0], [0, -1]
])

# 1. PauliWords DataStructure

In [5]:
# PauliWords: 1.0 XX + 1.0 XY + 1.0 XI
# PauliWord: 1.0 XX
# PauliOp: X(qubit=1)

## 1.1 PauliOp

In [9]:
class PauliOp:
    def __init__(self, op_type: str, index: int):
        if op_type not in ["I", "X", "Y", "Z"]:
            raise ValueError(f"operator tpye: {op_type} is not allowed!")
        self.type = op_type # I, X, Y, Z
        self.index = index
        
    @property
    def matrix(self) -> npmatrix:
        if self.type == "I":
            return I
        elif self.type == "X":
            return X
        elif self.type == "Y":
            return Y
        elif self.type == "Z":
            return Z
        
    def __str__(self) -> str:
        return f"{self.type} (qubit={self.index})"
    
    def __repr__(self) -> str:
        return f"{self.type} (qubit={self.index})"

In [10]:
# Test - PauliOp

x1 = PauliOp('X', 1)
print(x1)
print(x1.matrix)

X (qubit=1)
[[0 1]
 [1 0]]


## 1.2 PauliWord

In [14]:
class PauliWord:
    def __init__(self, op_type_str: str, coeff: complex = 1.0):
        self.num_qubits = len(op_type_str)
        self.op_type_str = op_type_str
        self.ops = []
        for idx, op_type in enumerate(op_type_str):
            self.ops.append(PauliOp(op_type, idx))
        self.coeff = coeff
        
    @property
    def matrix(self) -> npmatrix:
        ops = []
        for op in self.ops:
            ops.append(op.matrix)
        
        return self.coeff * Kron(*ops)
    
    def eliminate(self, eliminate_qubits_indexes: Union[int, List[int]]) -> None:
        if not isinstance(eliminate_qubits_indexes, list):
            eliminate_qubits_indexes = [eliminate_qubits_indexes]
        
        self.num_qubits -= len(eliminate_qubits_indexes)
        op_type_str = self.op_type_str
        op_type_arr = list(op_type_str)
        for index in eliminate_qubits_indexes:
            op_type_arr[index] = ""
        self.op_type_str = "".join(op_type_arr)
        
        ops = []
        for i, op in enumerate(self.ops):
            if i not in eliminate_qubits_indexes:
                ops.append(op)
        self.ops = ops
    
    def __mul__(self, other: PauliWord) -> PauliWord:
        if len(self.ops) != len(other.ops):
            raise ValueError("Different size PauliWord cannot be multiplied")
            
        coeff = self.coeff * other.coeff
        return_op_type_arr = []
        for op_l, op_r in zip(self.ops, other.ops):
            if op_l.type == op_r.type: # XX = I, YY = I, ZZ = I
                return_op_type_arr.append("I")
            elif op_l.type == "I": # IX = X
                return_op_type_arr.append(op_r.type)
            elif op_r.type == "I": # XI = X
                return_op_type_arr.append(op_l.type)
            elif op_l.type == "X" and op_r.type == "Y": # XY = iZ
                coeff = 1j * coeff
                return_op_type_arr.append("Z")
            elif op_l.type == "Y" and op_r.type == "X": # YX = -iZ
                coeff = -1j * coeff
                return_op_type_arr.append("Z")
            elif op_l.type == "X" and op_r.type == "Z": # XZ = -iY
                coeff = -1j * coeff
                return_op_type_arr.append("Y")
            elif op_l.type == "Z" and op_r.type == "X": # ZX = iY
                coeff = 1j * coeff
                return_op_type_arr.append("Y")
            elif op_l.type == "Y" and op_r.type == "Z": # YZ = iX
                coeff = 1j * coeff
                return_op_type_arr.append("X")
            elif op_l.type == "Z" and op_r.type == "Y": # ZY = -iX
                coeff = -1j * coeff
                return_op_type_arr.append("X")
                
        return PauliWord(''.join(return_op_type_arr), coeff)
        
    def __str__(self) -> str:
        pauli_word = [ op.type for op in self.ops ]
        return f"{self.coeff:.8f} {''.join(pauli_word)}"
    
    def __repr__(self) -> str:
        pauli_word = [ op.type for op in self.ops ]
        return f"{self.coeff:.8f} {''.join(pauli_word)}"

In [15]:
# Test - PauliWord

pauliword_IX = PauliWord("IX")
print(pauliword_IX)
print(pauliword_IX.ops[1])
print(pauliword_IX.matrix)

1.00000000 IX
X (qubit=1)
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [16]:
# Test - PauliWord __mul__

pauliword_XZ = PauliWord("XZ")
pauliword_YX = PauliWord("YX")

pauliword_XZ * pauliword_YX # should be -ZY

-1.00000000+0.00000000j ZY

## 1.3 PauliWords

In [17]:
from collections import defaultdict

class PauliWords:
    def __init__(self, op_type_strs: List[str], coeffs: Optional[List[complex]] = None):
        if coeffs != None and len(op_type_strs) != len(coeffs):
            raise ValueError("size of coeffs and size of op_type_strs should be the same!")
        if len(op_type_strs) == 0:
            raise ValueError("op_type_strs shouldn't be empty!")
        
        self.num_terms = len(op_type_strs)
        self.num_qubits = len(op_type_strs[0])
        if coeffs == None:
            coeffs = [1.0] * self.num_terms
        self.terms = []
        for coeff, op_type_str in zip(coeffs, op_type_strs):
            self.terms.append(PauliWord(op_type_str, coeff))
            
    @property
    def matrix(self) -> npmatrix:
        sub_hamis = [ term.matrix for term in self.terms ]
        return sum(sub_hamis)
    
    def simplify(self) -> PauliWords:
        term_dict = defaultdict(complex)
        for term in self.terms:
            term_dict[term.op_type_str] += term.coeff
            
        terms = []
        coeffs = []
        term_dict = { k:v for k,v in term_dict.items() if not abs(v) < 1e-10 }
        for k, v in term_dict.items():
            terms.append(PauliWord(k, v))
            coeffs.append(v)
            
        self.terms = terms
        self.coeffs = coeffs
        self.num_terms = len(terms)
            
        return self
    
    def eliminate(self, eliminate_qubits_indexes: Union[int, List[int]]) -> None:
        if not isinstance(eliminate_qubits_indexes, list):
            eliminate_qubits_indexes = [eliminate_qubits_indexes]
        
        self.num_qubits -= len(eliminate_qubits_indexes)
        for term in self.terms:
            term.eliminate(eliminate_qubits_indexes)
    
    def __mul__(self, other: PauliWords) -> PauliWords:
        terms = []
        for term_l in self.terms:
            for term_r in other.terms:
                terms.append(term_l * term_r)
        
        op_type_strs = []
        coeffs = []
        for term in terms:
            op_type_strs.append(term.op_type_str)
            coeffs.append(term.coeff)
            
        return PauliWords(op_type_strs, coeffs)
            
    def __str__(self) -> str:
        returns = []
        for pauliword in self.terms:
            returns.append(str(pauliword))
        return "\n".join(returns)
            
    def __repr__(self) -> str:
        returns = []
        for pauliword in self.terms:
            returns.append(repr(pauliword))
        return "\n".join(returns)

In [18]:
# Test - PauliWords

Hami1 = PauliWords(["XI", "YX"])
Hami2 = PauliWords(["XI", "YX", "YY"])
Hami3 = PauliWords(["XI", "YY"])

print(f"Hami1: \n{Hami1}\n")
print(f"Hami2: \n{Hami2}\n")
print(f"Hami3: \n{Hami3}")

Hami1: 
1.00000000 XI
1.00000000 YX

Hami2: 
1.00000000 XI
1.00000000 YX
1.00000000 YY

Hami3: 
1.00000000 XI
1.00000000 YY


In [19]:
# Test - PauliWords __mul__

A = PauliWords(["IX", "XZ"], [1/(2**0.5), 1/(2**0.5)])
B = PauliWords(["XI", "YX"])

print(f"A: \n{A}\n")
print(f"B: \n{B}\n")
print(f"ABA: \n{A * B * A}")
print(f"\nSimplified result: \n{(A * B * A).simplify()}")

A: 
0.70710678 IX
0.70710678 XZ

B: 
1.00000000 XI
1.00000000 YX

ABA: 
0.50000000 XI
0.00000000-0.50000000j IY
0.50000000 YX
0.00000000-0.50000000j ZZ
0.00000000+0.50000000j IY
0.50000000 XI
0.00000000+0.50000000j ZZ
0.50000000-0.00000000j YX

Simplified result: 
1.00000000+0.00000000j XI
1.00000000+0.00000000j YX


# 2. Construct Binary Matrix G(Gx | Gz) and parity check matrix E

In [20]:
# https://arxiv.org/pdf/1701.08213.pdf

In [21]:
def print_G(G_x: nparray, G_z: nparray) -> None:
    row = len(G_x)
    col = len(G_x[0])
    G_str = ''
    
    for r in range(row):
        for c in range(col):
            G_str += f"  {int(G_x[r][c])}"
        G_str += '\n'
    G_str += ' ' + '-' * 3 * col + '\n'
    
    for r in range(row):
        for c in range(col):
            G_str += f"  {int(G_z[r][c])}"
        G_str += '\n'
        
    print(G_str)

In [22]:
def print_E(E: nparray) -> None:
    row = len(E)
    col = len(E[0])
    E_str = ''
    
    for r in range(row):
        for c in range(col):
            if c == col // 2:
                E_str += "  |"
            E_str += f"  {int(E[r][c])}"
        E_str += '\n'
        
    print(E_str)

## 2.1 Binary Matrix G(Gx, Gz)

In [23]:
def create_binary_matrix_G(pauli_words: PauliWords) -> Tuple[nparray, nparray]:
    if not isinstance(pauli_words, PauliWords):
        raise ValueError("input should be a PauliWords instance")
    
    # size of Gx / Gz is (num_qubits, num_terms)
    num_terms = pauli_words.num_terms
    num_qubits = pauli_words.num_qubits
    G_x = np.zeros((num_qubits, num_terms))
    G_z = np.zeros((num_qubits, num_terms))
    
    for col_idx, term in enumerate(pauli_words.terms):
        for row_idx, op in enumerate(term.ops):
            if op.type == 'X':
                G_x[row_idx][col_idx] = 1
            elif op.type == 'Y':
                G_x[row_idx][col_idx] = 1
                G_z[row_idx][col_idx] = 1
            elif op.type == 'Z':
                G_z[row_idx][col_idx] = 1
    
    return G_x, G_z

In [24]:
# Test - create_binary_matrix_G

G_x1, G_z1 = create_binary_matrix_G(Hami1)
print(f"G of Hami1: ")
print_G(G_x1, G_z1)

G_x2, G_z2 = create_binary_matrix_G(Hami2)
print(f"G of Hami2: ")
print_G(G_x2, G_z2)

G_x3, G_z3 = create_binary_matrix_G(Hami3)
print(f"G of Hami3: ")
print_G(G_x3, G_z3)

G of Hami1: 
  1  1
  0  1
 ------
  0  1
  0  0

G of Hami2: 
  1  1  1
  0  1  1
 ---------
  0  1  1
  0  0  1

G of Hami3: 
  1  1
  0  1
 ------
  0  1
  0  1



## 2.2 parity check matrix

In [25]:
def create_parity_check_matrix_E(G_x: nparray, G_z: nparray) -> nparray:
    E_x = G_z.T
    E_z = G_x.T
    
    return np.hstack((E_x, E_z))

In [26]:
# Test - createParityCheckMatrixE

G_x1, G_z1 = create_binary_matrix_G(Hami1)
E1 = create_parity_check_matrix_E(G_x1, G_z1)
print("E1: ")
print_E(E1)

G_x2, G_z2 = create_binary_matrix_G(Hami2)
E2 = create_parity_check_matrix_E(G_x2, G_z2)
print("E2: ")
print_E(E2)

G_x3, G_z3 = create_binary_matrix_G(Hami3)
E3 = create_parity_check_matrix_E(G_x3, G_z3)
print("E3: ")
print_E(E3)

E1: 
  0  0  |  1  0
  1  0  |  1  1

E2: 
  0  0  |  1  0
  1  0  |  1  1
  1  1  |  1  1

E3: 
  0  0  |  1  0
  1  1  |  1  1



# 3. kernel calculation and create Generators

In [27]:
# E --(need: Gauss Jordan elimination)--> kernel(E)
#   --> generators --> paulix_ops

## 3.0 Gauss Jordan elimination

In [153]:
# https://github.com/potassco/xorro

In [28]:
def remove_zeros_rows(m: nparray) -> List[nparray]:
    return_m = []
    for row in m:
        if sum(row) > 0:
            return_m.append(row)
    
    return return_m

In [29]:
# Test - remove_zeros_rows

test_matrix = np.array([
    [1, 1, 0, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
])
print(remove_zeros_rows(test_matrix))

[array([1, 1, 0, 0, 0]), array([1, 0, 0, 1, 0]), array([0, 1, 0, 1, 0])]


In [30]:
def xor(m: nparray, i: int, j: int) -> nparray:
    for k in range(len(m[0])):
        m[j][k] ^= m[i][k]
    
    return m

In [31]:
def perform_gauss_jordan_elimination(m: List[nparray]) -> nparray:
    dimension = len(m)
    
    # 1. Forward Elimination
    r = 0
    right_most_col = 0
    lowest_row = 0
    
    for c in range(len(m[0]) - 1):
        _swap = False
        _xor  = False
        for j in range(r + 1, dimension):
            if m[r][c] == 0 and m[j][c] == 1:
                m[r], m[j] = m[j], m[r]
                _swap = True
            if m[r][c] == 1:
                _xor = True
                if m[j][c] == 1:
                    m = xor(m, r, j)

        if m[r][c] == 1:
            right_most_col = c
            lowest_row = r
        if _swap or _xor:
            r += 1

    # 2. Backward Substitution
    r = lowest_row
    for c in range(right_most_col, 0, -1):
        _xor = False
        
        for j in range(r - 1, -1, -1):
            if m[r][c] == 1 and m[j][c] == 1:
                _xor = True
                m = xor(m, r, j)
                    
        if m[r][c - 1] == 0:
            r -= 1

    return m

In [32]:
def solve_GJE(m: nparray) -> nparray:
    if len(m[0]) > 2:
        m = remove_zeros_rows(m)
        m = perform_gauss_jordan_elimination(m)
        
    return m

In [33]:
# Test - solve_GJE

test_matrix = np.array([
    [1, 1, 0, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0]
])
print(f'Before GJE: \n{test_matrix}\n')
print(f'After GJE: \n{solve_GJE(test_matrix)}')

Before GJE: 
[[1 1 0 0 0]
 [1 0 0 1 0]
 [0 1 0 1 0]
 [0 0 1 0 0]]

After GJE: 
[array([1, 0, 0, 1, 0]), array([0, 1, 0, 1, 0]), array([0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0])]


## 3.1 calculate kernel of E

In [34]:
from sympy import Matrix

def kernel_of_E(E: nparray) -> nparray:
    E_ = solve_GJE(E.astype(int))
    E_ = Matrix(E_)
    kernel_vectors = []
    
    for vector in E_.nullspace():
        kernel_vectors.append(
            abs(np.array(vector)) 
                .flatten()
                .astype(int)
        ) # => eg. [1, 0, 0, 1]
    
    return np.array(kernel_vectors)

In [35]:
# Test - kernel_of_E

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
kernel_vectors1 = kernel_of_E(E1)
print("Kernel of E1: \n", kernel_vectors1)

E2 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami2))
kernel_vectors2 = kernel_of_E(E2)
print("\nKernel of E2: \n", kernel_vectors2)

E3 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami3))
kernel_vectors3 = kernel_of_E(E3)
print("\nKernel of E3: \n", kernel_vectors3)

Kernel of E1: 
 [[0 1 0 0]
 [1 0 0 1]]

Kernel of E2: 
 [[1 0 0 1]]

Kernel of E3: 
 [[1 1 0 0]
 [1 0 0 1]]


## 3.2 kernel => generators

In [36]:
def get_generator_from_kernel(kernel_vector: nparray) -> PauliWord:
    num_qubits = len(kernel_vector) // 2
    
    op_type_arr = []
    for qubit_idx in range(num_qubits):
        if kernel_vector[qubit_idx] == 1 and kernel_vector[qubit_idx + num_qubits] == 1:
            op_type_arr.append("Y")
        elif kernel_vector[qubit_idx] == 1:
            op_type_arr.append("X")
        elif kernel_vector[qubit_idx + num_qubits] == 1:
            op_type_arr.append("Z")
        else:
            op_type_arr.append("I")
    op_type_str = ''.join(op_type_arr)
    
    return PauliWord(op_type_str)

In [37]:
# Test - get_generator_from_kernel

kernel_vector = np.array([1, 0, 0, 1])
print("kernel: (1, 0, | 0, 1) => ", get_generator_from_kernel(kernel_vector))

kernel: (1, 0, | 0, 1) =>  1.00000000 XZ


In [38]:
def get_generators_from_kernel(kernel_vectors: nparray) -> PauliWords:
    if len(kernel_vectors) == 0:
        raise ValueError("input kernel is empty!")
        
    generators = [ get_generator_from_kernel(kernel_vector).op_type_str 
                  for kernel_vector in kernel_vectors ]
    
    return PauliWords(generators)

In [93]:
# Test - get_generators_from_kernel

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1)) # generate binary matrix G -> parity check matrix E
generators1 = get_generators_from_kernel(kernel_of_E(E1)) # parity check matrix -> generators
print("Generators of Hami1: \n", generators1)

E2 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami2)) # generate binary matrix G -> parity check matrix E
generators2 = get_generators_from_kernel(kernel_of_E(E2)) # parity check matrix -> generators
print("\nGenerators of Hami2: \n", generators2)

E3 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami3)) # generate binary matrix G -> parity check matrix E
generators3 = get_generators_from_kernel(kernel_of_E(E3)) # parity check matrix -> generators
print("\nGenerators of Hami3: \n", generators3)

Generators of Hami1: 
 1.00000000 IX
1.00000000 XZ

Generators of Hami2: 
 1.00000000 XZ

Generators of Hami3: 
 1.00000000 XX
1.00000000 XZ


## 3.3 generators => corresponding paulixop

### - tool functions to check commutes or anti-commutes property

In [219]:
def is_commutes(generator: PauliWord, paulix_index: int) -> bool:
    # paulix_op: III...X..II
    # generator: ???...△..II
    # only need to check X△ = △X => △ = 'X' or 'I'
    return generator.ops[paulix_index].type == 'X' or \
        generator.ops[paulix_index].type == 'I'

In [220]:
def is_anti_commutes(generator: PauliWord, paulix_index: int) -> bool:
    # paulix_op: III...X..II
    # generator: ???...△..II
    # only need to check X△ = -△X => △ = 'Z' or 'Y'
    return generator.ops[paulix_index].type == 'Z' or \
        generator.ops[paulix_index].type == 'Y'

### - get paulix_op from generator

In [223]:
def get_paulix_op_from_generator(i: int, generator: PauliWord, generators: PauliWords) -> Tuple[bool, Optional[PauliWord]]:
    for op in reversed(generator.ops):
        if op.type != 'I':
            op_type_arr = ["I"] * generator.num_qubits
            op_type_arr[op.index] = "X"
            paulix_op = PauliWord(''.join(op_type_arr))
            
            # X_q(i) anti-commutes with tau_i
            if not is_anti_commutes(generator, op.index): continue
            # X_q(i) commutes with tau_j (j≠i)
            for j, generator_j in enumerate(generators.terms):
                if j != i:
                    if not is_commutes(generator_j, op.index): continue
            return True, paulix_op
    
    return False, None

In [224]:
# Test - get_paulix_op_from_generator

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
generators1 = get_generators_from_kernel(kernel_of_E(E1))

generator1 = generators1.terms[0] # IX
print('Generator1: ', generator1)
print("paulix_op: ", get_paulix_op_from_generator(0, generator1, generators1))
generator2 = generators1.terms[1] # XZ
print('\nGenerator2: ', generator2)
print("paulix_op: ", get_paulix_op_from_generator(1, generator2, generators1))

Generator1:  1.00000000 IX
paulix_op:  (False, None)

Generator2:  1.00000000 XZ
paulix_op:  (True, 1.00000000 IX)


In [44]:
def get_paulix_ops_from_generators(generators: PauliWords) -> Tuple[PauliWords, PauliWords]:
    rechecked_generators = []
    paulix_ops = []
    
    for i, generator in enumerate(generators.terms):
        flag, paulix_op = get_paulix_op_from_generator(i, generator, generators)
        if flag:
            rechecked_generators.append(generator.op_type_str)
            paulix_ops.append(paulix_op.op_type_str)
    
    return PauliWords(rechecked_generators), PauliWords(paulix_ops)

In [94]:
# Test - getPauliXOpFromGenerators

# generate binary matrix G -> parity check matrix E -> generators
E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1)) 
generators1 = get_generators_from_kernel(kernel_of_E(E1))
# generators -> paulix_ops
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(generators1)
print("Generators of Hami1: \n", generators1)
print("Rechecked Generators of Hami1: \n", rechecked_generators1)
print("and corresponding paulixOps: \n", paulix_ops1)

# generate binary matrix G -> parity check matrix E -> generators
E2 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami2))
generators2 = get_generators_from_kernel(kernel_of_E(E2))
# generators -> paulix_ops
rechecked_generators2, paulix_ops2 = get_paulix_ops_from_generators(generators2)
print("\nGenerators of Hami2: \n", generators2)
print("Rechecked Generators of Hami2: \n", rechecked_generators2)
print("and corresponding paulixOps: \n", paulix_ops2)

# generate binary matrix G -> parity check matrix E -> generators
E3 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami3))
generators3 = get_generators_from_kernel(kernel_of_E(E3))
# generators -> paulix_ops
rechecked_generators3, paulix_ops3 = get_paulix_ops_from_generators(generators3)
print("\nGenerators of Hami3: \n", generators3)
print("Rechecked Generators of Hami3: \n", rechecked_generators3)
print("and corresponding paulixOps: \n", paulix_ops3)

Generators of Hami1: 
 1.00000000 IX
1.00000000 XZ
Rechecked Generators of Hami1: 
 1.00000000 XZ
and corresponding paulixOps: 
 1.00000000 IX

Generators of Hami2: 
 1.00000000 XZ
Rechecked Generators of Hami2: 
 1.00000000 XZ
and corresponding paulixOps: 
 1.00000000 IX

Generators of Hami3: 
 1.00000000 XX
1.00000000 XZ
Rechecked Generators of Hami3: 
 1.00000000 XZ
and corresponding paulixOps: 
 1.00000000 IX


# 4. Construct unitary matrix U

In [96]:
# generators + paulix_ops    optimal sector
#            ↘                   ↙
#       unitary matrix U     adjusted Hamilton
#              ↘             ↙
#                  H' = U†HU

In [47]:
def construct_U(generators: PauliWords, paulix_ops: PauliWords) -> List[PauliWords]:
    c = 1 / (2 ** 0.5) 
    Us = []
    
    for generator, paulix_op in zip(generators.terms, paulix_ops.terms):
        Us.append(PauliWords(
            [paulix_op.op_type_str, generator.op_type_str],
            [c * paulix_op.coeff, c * generator.coeff]
        ))
        
    return Us

In [48]:
# Test - construct_U

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
generators1 = get_generators_from_kernel(kernel_of_E(E1))
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(generators1)
construct_U(rechecked_generators1, paulix_ops1)

[0.70710678 IX
 0.70710678 XZ]

## 4.1 Adjust Hamilton by optimal sector

In [50]:
# eg. H: XYYX, paulix_ops: IXII IIXI IIIX, sectors: [1, -1, -1]
# XYYX --(applying: IXII IIXI)--> [1, -1] -> -XYYX

### - get optimal sector

In [51]:
def optimal_sector(Hami: PauliWords, generators: PauliWords, active_electrons: int) -> List[int]:
    num_orbitals = Hami.num_qubits

    if active_electrons > num_orbitals:
        raise ValueError(
            f"Number of active orbitals cannot be smaller than number of active electrons;"
            f" got 'orbitals'={num_orbitals} < 'electrons'={active_electrons}."
        )

    hf_str = np.where(np.arange(num_orbitals) < active_electrons, 1, 0)

    perm = []
    for generator in generators.terms:
        symmstr = np.array([1 if generator.ops[qubit].type != 'I' else 0 for qubit in range(Hami.num_qubits)])
        coeff = -1 if np.logical_xor.reduce(np.logical_and(symmstr, hf_str)) else 1
        perm.append(coeff)

    return perm

In [52]:
# Test - optimal_sector

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
generators1 = get_generators_from_kernel(kernel_of_E(E1))
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(generators1)
optimal_sector(Hami1, rechecked_generators1, 2)

[1]

### - pauli x index of paulix_ops

In [53]:
def get_paulix_op_indexes(paulix_ops: PauliWords) -> List[int]:
    x_indexes = []
    
    for paulix_op in paulix_ops.terms:
        for op in paulix_op.ops:
            if op.type == "X":
                x_indexes.append(op.index)
                break
    
    return x_indexes

In [55]:
# Test - get_paulix_op_indexes

E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
generators1 = get_generators_from_kernel(kernel_of_E(E1))
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(generators1)
x_indexes1 = get_paulix_op_indexes(paulix_ops1)
print("paulixOps: \n", paulix_ops1)
print("x_indexes: ", x_indexes1)

paulixOps: 
 1.00000000 IX
x_indexes:  [1]


### - adjust hamilton by optimal sector

In [56]:
def adjust_hamilton_by_optimal_sector(Hami: PauliWords, paulix_ops: PauliWords, optimal_sector: List[int]) -> PauliWords:
    x_indexes = get_paulix_op_indexes(paulix_ops)
    
    terms = []
    coeffs = []
    for i, term in enumerate(Hami.terms):
        terms.append(term.op_type_str)
        coeff = term.coeff
        
        for x_index, sector in zip(x_indexes, optimal_sector):
            if term.ops[x_index].type != "I" and term.ops[x_index].type != "X":
                coeff *= sector
        
        coeffs.append(coeff)
            
    return PauliWords(terms, coeffs)

## 4.2 calculate H' = U†HU

In [58]:
def unitary_transform(Us: List[PauliWords], Hami: PauliWords) -> PauliWords:
    H_prime = Hami
    
    for U in Us:
        H_prime = U * H_prime * U
        H_prime = H_prime.simplify()
        
    return H_prime

In [98]:
# Test - unitary_transform

# binary matrix G -> parity check matrix -> generators & paulix_ops
E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E1)))
# generators + n_electrons -> optimal sector
optimal_sector1 = optimal_sector(Hami1, rechecked_generators1, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami1_prime = unitary_transform(
    construct_U(rechecked_generators1, paulix_ops1), 
    adjust_hamilton_by_optimal_sector(Hami1, paulix_ops1, optimal_sector1))
print("Hami1_prime: \n", Hami1_prime)

# binary matrix G -> parity check matrix -> generators & paulix_ops
E2 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami2))
rechecked_generators2, paulix_ops2 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E2)))
# generators + n_electrons -> optimal sector
optimal_sector2 = optimal_sector(Hami2, rechecked_generators2, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami2_prime = unitary_transform(
    construct_U(rechecked_generators2, paulix_ops2), 
    adjust_hamilton_by_optimal_sector(Hami2, paulix_ops2, optimal_sector2))
print("\nHami2_prime: \n", Hami2_prime)

# binary matrix G -> parity check matrix -> generators & paulix_ops
E3 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami3))
rechecked_generators3, paulix_ops3 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E3)))
# generators + n_electrons -> optimal sector
optimal_sector3 = optimal_sector(Hami3, rechecked_generators3, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami3_prime = unitary_transform(
    construct_U(rechecked_generators3, paulix_ops3), 
    adjust_hamilton_by_optimal_sector(Hami3, paulix_ops3, optimal_sector3))
print("\nHami3_prime: \n", Hami3_prime)

Hami1_prime: 
 1.00000000+0.00000000j XI
1.00000000+0.00000000j YX

Hami2_prime: 
 1.00000000+0.00000000j XI
1.00000000+0.00000000j YX
1.00000000+0.00000000j ZI

Hami3_prime: 
 1.00000000+0.00000000j XI
1.00000000+0.00000000j ZI


# 5. Eliminate extra qubits H' => H''

In [99]:
def eliminate_qubits(Hami_prime: PauliWords, paulix_ops: PauliWords) -> PauliWords:
    eliminate_qubits_indexes = []
    for paulix_op in paulix_ops.terms:
        for op in paulix_op.ops:
            if op.type == 'X':
                eliminate_qubits_indexes.append(op.index)
                break
    
    Hami_prime.eliminate(eliminate_qubits_indexes)
    
    return Hami_prime.simplify()

In [100]:
# Test - eliminate_qubits

# binary matrix G -> parity check matrix -> generators & paulix_ops
E1 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami1))
rechecked_generators1, paulix_ops1 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E1)))
# generators + n_electrons -> optimal sector
optimal_sector1 = optimal_sector(Hami1, rechecked_generators1, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami1_prime = unitary_transform(
    construct_U(rechecked_generators1, paulix_ops1), 
    adjust_hamilton_by_optimal_sector(Hami1, paulix_ops1, optimal_sector1))
# H'' = H' after eliminating extra qubits
Hami1_prime_eliminated = eliminate_qubits(Hami1_prime, paulix_ops1)
print("Hami1'': \n", Hami1_prime_eliminated)

# binary matrix G -> parity check matrix -> generators & paulix_ops
E2 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami2))
rechecked_generators2, paulix_ops2 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E2)))
# generators + n_electrons -> optimal sector
optimal_sector2 = optimal_sector(Hami2, rechecked_generators2, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami2_prime = unitary_transform(
    construct_U(rechecked_generators2, paulix_ops2), 
    adjust_hamilton_by_optimal_sector(Hami2, paulix_ops2, optimal_sector2))
# H'' = H' after eliminating extra qubits
Hami2_prime_eliminated = eliminate_qubits(Hami2_prime, paulix_ops2)
print("\nHami2'': \n", Hami2_prime_eliminated)

# binary matrix G -> parity check matrix -> generators & paulix_ops
E3 = create_parity_check_matrix_E(*create_binary_matrix_G(Hami3))
rechecked_generators3, paulix_ops3 = get_paulix_ops_from_generators(
    get_generators_from_kernel(kernel_of_E(E3)))
# generators + n_electrons -> optimal sector
optimal_sector3 = optimal_sector(Hami3, rechecked_generators3, 2)
# H' = U†HU (H has adjusted by optimal sector)
Hami3_prime = unitary_transform(
    construct_U(rechecked_generators3, paulix_ops3), 
    adjust_hamilton_by_optimal_sector(Hami3, paulix_ops3, optimal_sector3))
# H'' = H' after eliminating extra qubits
Hami3_prime_eliminated = eliminate_qubits(Hami3_prime, paulix_ops3)
print("\nHami3'': \n", Hami3_prime_eliminated)

Hami1'': 
 1.00000000+0.00000000j X
1.00000000+0.00000000j Y

Hami2'': 
 1.00000000+0.00000000j X
1.00000000+0.00000000j Y
1.00000000+0.00000000j Z

Hami3'': 
 1.00000000+0.00000000j X
1.00000000+0.00000000j Z


# 6. H2 example

In [101]:
# 1. construct origin Hamilton
# 2. generate binary matrix G(Gx|Gz) and parity check matrix E
# 3. get generators and corresponding paulix_ops using kernel(E)
# 4. get optimal sector and adjust Hamilton
# 5. construct unitary U, and get H' = U†HU
# 6. eliminated extra qubits

## 6.1 construct origin Hamilton

In [102]:
# -0.042072551947 IIII
#  0.177713582291 ZIII
#  0.177713582291 IZII
#  0.170597592768 ZZII
#  0.044750084063 YXXY
# -0.044750084063 YYXX
# -0.044750084063 XXYY
#  0.044750084063 XYYX
# -0.242745012609 IIZI
#  0.122933304493 ZIZI
# -0.242745012609 IIIZ
#  0.167683388556 ZIIZ
#  0.167683388556 IZZI
#  0.122933304493 IZIZ
#  0.176276613942 IIZZ

In [103]:
Hami_H2 = PauliWords(
    ["IIII", "ZIII", "IZII", "ZZII", "YXXY", "YYXX", 
     "XXYY", "XYYX", "IIZI", "ZIZI", "IIIZ", "ZIIZ", 
     "IZZI", "IZIZ", "IIZZ"],
    [-0.042072551947, 0.177713582291, 0.177713582291, 0.170597592768, 
     0.044750084063, -0.044750084063, -0.044750084063, 0.044750084063, 
     -0.242745012609, 0.122933304493, -0.242745012609, 0.167683388556,
     0.167683388556, 0.122933304493, 0.176276613942  ]
)

## 6.2 generate binary matrix G(Gx|Gz) and parity check matrix E

In [104]:
G_x_H2, G_z_H2 = create_binary_matrix_G(Hami_H2)
print('G_x:')
print_G(G_x_H2, G_z_H2)

E_H2 = create_parity_check_matrix_E(G_x_H2, G_z_H2)
print('E:')
print_E(E_H2)

G_x:
  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  0  0  0  0  0  0  0
 ---------------------------------------------
  0  1  0  1  1  1  0  0  0  1  0  1  0  0  0
  0  0  1  1  0  1  0  1  0  0  0  0  1  1  0
  0  0  0  0  0  0  1  1  1  1  0  0  1  0  1
  0  0  0  0  1  0  1  0  0  0  1  1  0  1  1

E:
  0  0  0  0  |  0  0  0  0
  1  0  0  0  |  0  0  0  0
  0  1  0  0  |  0  0  0  0
  1  1  0  0  |  0  0  0  0
  1  0  0  1  |  1  1  1  1
  1  1  0  0  |  1  1  1  1
  0  0  1  1  |  1  1  1  1
  0  1  1  0  |  1  1  1  1
  0  0  1  0  |  0  0  0  0
  1  0  1  0  |  0  0  0  0
  0  0  0  1  |  0  0  0  0
  1  0  0  1  |  0  0  0  0
  0  1  1  0  |  0  0  0  0
  0  1  0  1  |  0  0  0  0
  0  0  1  1  |  0  0  0  0



## 6.3 get generators and corresponding paulix_ops using kernel(E)

In [105]:
kernel_vectors_H2 = kernel_of_E(E_H2)
print('Kernel: \n', kernel_vectors_H2)

Kernel: 
 [[0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 1 0 0 1]]


In [106]:
generators_H2 = get_generators_from_kernel(kernel_vectors_H2)
print('Generators: \n', generators_H2)

rechecked_generators_H2, paulix_ops_H2 = get_paulix_ops_from_generators(generators_H2)
print('\nRechecked generators: \n', rechecked_generators_H2)
print('\npaulix_ops: \n', paulix_ops_H2)

Generators: 
 1.00000000 ZZII
1.00000000 ZIZI
1.00000000 ZIIZ

Rechecked generators: 
 1.00000000 ZZII
1.00000000 ZIZI
1.00000000 ZIIZ

paulix_ops: 
 1.00000000 IXII
1.00000000 IIXI
1.00000000 IIIX


## 6.4 get optimal sector and adjust Hamilton

In [107]:
# optimal sector
n_electrons = 2
optimal_sector_H2 = optimal_sector(Hami_H2, rechecked_generators_H2, n_electrons)
print('optimal sector: ', optimal_sector_H2)

optimal sector:  [1, -1, -1]


In [108]:
Hami_H2_adjusted = adjust_hamilton_by_optimal_sector(Hami_H2, paulix_ops_H2, optimal_sector_H2)
Hami_H2_adjusted

-0.04207255 IIII
0.17771358 ZIII
0.17771358 IZII
0.17059759 ZZII
-0.04475008 YXXY
-0.04475008 YYXX
-0.04475008 XXYY
-0.04475008 XYYX
0.24274501 IIZI
-0.12293330 ZIZI
0.24274501 IIIZ
-0.16768339 ZIIZ
-0.16768339 IZZI
-0.12293330 IZIZ
0.17627661 IIZZ

## 6.5 construct unitary U, and get H' = U†HU

In [109]:
Us_H2 = construct_U(rechecked_generators_H2, paulix_ops_H2)
Hami_H2_prime = unitary_transform(Us_H2, Hami_H2_adjusted)
print("Hami_H2_prime: \n", Hami_H2_prime)

Hami_H2_prime: 
 -0.04207255+0.00000000j IIII
0.17771358+0.00000000j ZIII
0.17771358+0.00000000j ZXII
0.17059759+0.00000000j IXII
0.04475008+0.00000000j XXXI
0.04475008+0.00000000j XIXX
0.04475008+0.00000000j XXII
0.04475008+0.00000000j XIIX
0.24274501+0.00000000j ZIXI
-0.12293330+0.00000000j IIXI
0.24274501+0.00000000j ZIIX
-0.16768339+0.00000000j IIIX
-0.16768339+0.00000000j IXXI
-0.12293330+0.00000000j IXIX
0.17627661+0.00000000j IIXX


## 6.6 eliminated extra qubits

In [110]:
Hami_H2_prime_eliminated = eliminate_qubits(Hami_H2_prime, paulix_ops_H2)
print("Hami_H2_prime_eliminated: \n", Hami_H2_prime_eliminated)

Hami_H2_prime_eliminated: 
 -0.27643173+0.00000000j I
0.84091719+0.00000000j Z
0.17900034+0.00000000j X


In [111]:
# -0.276431731335 I
#  0.840917189801 Z
#  0.179000336252 X

## 6.7 eigenvalues results

In [112]:
E_fci = -1.136189454088

print('Test: original Hamilton\n')
print("Sorted Eigenvalues: ")
eigvals_ori = np.linalg.eigvals(Hami_H2.matrix)
print(np.sort(eigvals_ori))
print('E_fci: ', E_fci)
print('min eigenvalue: ', min(eigvals_ori))

print('-' * 20)
print('Test: Hami_reduced\n')
print("Sorted Eigenvalues: ")
eigvals_reduced = np.linalg.eigvals(Hami_H2_prime_eliminated.matrix)
print(np.sort(eigvals_reduced))
print('E_fci: ', E_fci)
print('min eigenvalue: ', min(eigvals_reduced))

Test: original Hamilton

Sorted Eigenvalues: 
[-1.13618916+0.j -0.52188356+0.j -0.52188356+0.j -0.47844693+0.j
 -0.47844693+0.j -0.47844693+0.j -0.40317874+0.j -0.40317874+0.j
 -0.12044625+0.j  0.30767559+0.j  0.30767559+0.j  0.44909649+0.j
  0.44909649+0.j  0.5833257 +0.j  0.75597218+0.j  1.0160979 +0.j]
E_fci:  -1.136189454088
min eigenvalue:  (-1.1361891625209464+0j)
--------------------
Test: Hami_reduced

Sorted Eigenvalues: 
[-1.13618916+0.j  0.5833257 +0.j]
E_fci:  -1.136189454088
min eigenvalue:  (-1.1361891625209464+0j)


# 7. LiH example

In [113]:
# 1. construct origin Hamilton
# 2. generate binary matrix G(Gx|Gz) and parity check matrix E
# 3. get generators and corresponding paulix_ops using kernel(E)
# 4. get optimal sector and adjust Hamilton
# 5. construct unitary U, and get H' = U†HU
# 6. eliminated extra qubits

## 7.1 construct origin Hamilton

In [114]:
import pennylane as qml
from pennylane import qchem

In [129]:
symbols_LiH = ["Li", "H"]
coordinates_LiH = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.8])

In [130]:
# 基于JW变换构建的H2哈密顿量
H_LiH, qubits_LiH = qml.qchem.molecular_hamiltonian(symbols_LiH, coordinates_LiH, basis="sto-3g")
print("Number of qubits = ", qubits_LiH)
print("The Hamiltonian is ", H_LiH)

Number of qubits =  12
The Hamiltonian is    (-2.4639981250646237) [I0]
+ (-0.4446198478205301) [Z11]
+ (-0.44461984782053) [Z10]
+ (-0.20614322861727225) [Z8]
+ (-0.2061432286172722) [Z9]
+ (-0.20614322861720352) [Z6]
+ (-0.20614322861720344) [Z7]
+ (-0.17798299728079822) [Z4]
+ (-0.17798299728079822) [Z5]
+ (-0.12622222084858362) [Z2]
+ (-0.12622222084858345) [Z3]
+ (1.2266687476181644) [Z0]
+ (1.2266687476181652) [Z1]
+ (-0.012184540179078163) [Y2 Y4]
+ (-0.012184540179078163) [X2 X4]
+ (8.999755369448373e-07) [Y1 Y3]
+ (8.999755369448373e-07) [X1 X3]
+ (0.0014789403289407206) [Y3 Y5]
+ (0.0014789403289407206) [X3 X5]
+ (0.0447291929263845) [Y0 Y2]
+ (0.0447291929263845) [X0 X2]
+ (0.06030173630501553) [Z2 Z4]
+ (0.06030173630501553) [Z3 Z5]
+ (0.06290858600785956) [Z2 Z6]
+ (0.06290858600785956) [Z3 Z7]
+ (0.06290858600788868) [Z2 Z8]
+ (0.06290858600788868) [Z3 Z9]
+ (0.06347928272861363) [Z4 Z6]
+ (0.06347928272861363) [Z5 Z7]
+ (0.06347928272864295) [Z4 Z8]
+ (0.0634792827286429

In [132]:
qml_generators_LiH = qml.symmetry_generators(H_LiH)
qml_paulixops_LiH = qml.paulix_ops(qml_generators_LiH, qubits_LiH)

for idx, generator in enumerate(qml_generators_LiH):
    print(f"generator {idx+1}: {generator}, paulix_op: {qml_generators_LiH[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op:   (1.0) [Z6 Z7]
generator 2:   (1.0) [Z8 Z9], paulix_op:   (1.0) [Z8 Z9]
generator 3:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10], paulix_op:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10]
generator 4:   (1.0) [Z1 Z3 Z5 Z6 Z8 Z11], paulix_op:   (1.0) [Z1 Z3 Z5 Z6 Z8 Z11]


In [133]:
n_electrons = 4
qml_paulix_sector_LiH = qml.qchem.optimal_sector(H_LiH, qml_generators_LiH, n_electrons)
print(qml_paulix_sector_LiH)

[1, 1, 1, 1]


In [135]:
H_tapered_LiH = qml.taper(H_LiH, qml_generators_LiH, qml_paulixops_LiH, qml_paulix_sector_LiH)
print(H_tapered_LiH)

  ((-2.3075253980220993+0j)) [I0]
+ ((-0.4122864572345443+0j)) [Z8]
+ ((-0.4122864572344067+0j)) [Z6]
+ ((-0.17798299728079806+0j)) [Z4]
+ ((-0.17798299728079806+0j)) [Z5]
+ ((-0.1262222208485835+0j)) [Z2]
+ ((-0.1262222208485833+0j)) [Z3]
+ ((-0.0022962614713492183+0j)) [X4]
+ ((0.0024935765657631516+0j)) [X6]
+ ((0.002493576565764307+0j)) [X8]
+ ((0.009473375730501346+0j)) [X0]
+ ((0.018467203459129826+0j)) [X5]
+ ((0.0720179243844852+0j)) [X1]
+ ((1.2266687476181632+0j)) [Z0]
+ ((1.2266687476181644+0j)) [Z1]
+ ((-0.03457860695898634+0j)) [Y0 Y1]
+ ((-0.022893202144134702+0j)) [Y2 Y3]
+ ((-0.01218454017907816+0j)) [Y2 Y4]
+ ((-0.01218454017907816+0j)) [X2 X4]
+ ((-0.008885075921644172+0j)) [X3 X4]
+ ((-0.007923228959415272+0j)) [Y4 Y5]
+ ((-0.0035777513967630774+0j)) [Y0 Y5]
+ ((-0.0010277015194125416+0j)) [X0 X8]
+ ((-0.0010277015194125416+0j)) [Y0 Y8]
+ ((-0.0010277015194120663+0j)) [X0 X6]
+ ((-0.0010277015194120663+0j)) [Y0 Y6]
+ ((8.999755369448369e-07+0j)) [Y1 Y3]
+ ((8.9997553

### - Hamiltonian to PauliWords

In [123]:
def switch_op(operator: qml.operation) -> Tuple[npmatrix, str]:
    try:
        name = operator.base_name
    except:
        name = operator
    
    if name == "Identity" or name == "I":
        op = I; label = "I"
    elif name == "PauliX" or name == "X":
        op = X; label = "X"
    elif name == "PauliY" or name == "Y":
        op = Y; label = "Y"
    elif name == "PauliZ" or name == "Z":
        op = Z; label = "Z"
        
    return op, label

In [124]:
def create_pauliwords_from_hamilton(H: qml.Hamiltonian, qubits: int) -> PauliWords:
    coeffs = []
    terms = []
    for i, op in enumerate(H.ops):
        op_labels = ['I' for i in range(qubits)]
        sub_Hami = [I for i in range(qubits)]

        if type(op) == qml.operation.Tensor:
            for ob in op.obs:
                operator, label = switch_op(ob)
                idx = int(ob.wires.labels[0])
                sub_Hami[idx] = operator
                op_labels[idx] = label
        else:
            operator, label = switch_op(op)
            idx = int(op.wires.labels[0])
            sub_Hami[idx] = operator
            op_labels[idx] = label
        
        coeff = H.coeffs[i]
        coeffs.append(coeff)
        print(f"{'-' if coeff < 0 else ' '}{abs(coeff):.12f} {''.join(op_labels)}")
        terms.append(''.join(op_labels))
    
    return PauliWords(terms, coeffs)

In [138]:
Hami_LiH = create_pauliwords_from_hamilton(H_LiH, qubits_LiH)

-2.463998125065 IIIIIIIIIIII
 1.226668747618 ZIIIIIIIIIII
 1.226668747618 IZIIIIIIIIII
 0.401860734590 ZZIIIIIIIIII
 0.019753539712 YZYIIIIIIIII
 0.044729192926 YIYIIIIIIIII
 0.019753539712 XZXIIIIIIIII
 0.044729192926 XIXIIIIIIIII
 0.001516492369 YZZZYIIIIIII
-0.002126287155 YIZZYIIIIIII
 0.001516492369 XZZZXIIIIIII
-0.002126287155 XIZZXIIIIIII
-0.042294705107 YZZZZZZZZZYI
-0.072017924384 YIZZZZZZZZYI
-0.042294705107 XZZZZZZZZZXI
-0.072017924384 XIZZZZZZZZXI
 0.019753539712 IYZYIIIIIIII
 0.044729192926 ZYZYIIIIIIII
 0.019753539712 IXZXIIIIIIII
 0.044729192926 ZXZXIIIIIIII
 0.011071097480 YXXYIIIIIIII
-0.011071097480 YYXXIIIIIIII
-0.011071097480 XXYYIIIIIIII
 0.011071097480 XYYXIIIIIIII
-0.001874590600 YXIXYIIIIIII
-0.001874590600 YYIYYIIIIIII
-0.001874590600 XXIXXIIIIIII
-0.001874590600 XYIYXIIIIIII
 0.019153618204 YXIXZZZZZZYI
 0.019153618204 YYIYZZZZZZYI
 0.019153618204 XXIXZZZZZZXI
 0.019153618204 XYIYZZZZZZXI
 0.001516492369 IYZZZYIIIIII
-0.002126287155 ZYZZZYIIIIII
 0.00151649236

In [137]:
Hami_LiH_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered_LiH, 9)

-2.307525398022 IIIIIIIII
 1.226668747618 ZIIIIIIII
 1.226668747618 IZIIIIIII
 0.401860734590 ZZIIIIIII
 0.019753539712 YZYIIIIII
 0.044729192926 YIYIIIIII
 0.019753539712 XZXIIIIII
 0.044729192926 XIXIIIIII
 0.001516492369 YZZZYIIII
-0.002126287155 YIZZYIIII
 0.001516492369 XZZZXIIII
-0.002126287155 XIZZXIIII
 0.042294705107 XZIZIZZIZ
 0.072017924384 XIIZIZZIZ
-0.042294705107 XZZZZZIII
-0.072017924384 XIZZZZIII
 0.019753539712 IYZYIIIII
 0.044729192926 ZYZYIIIII
 0.019753539712 IXZXIIIII
 0.044729192926 ZXZXIIIII
 0.011071097480 YXXYIIIII
-0.011071097480 YYXXIIIII
-0.011071097480 XXYYIIIII
 0.011071097480 XYYXIIIII
-0.001874590600 YXIXYIIII
-0.001874590600 YYIYYIIII
-0.001874590600 XXIXXIIII
-0.001874590600 XYIYXIIII
-0.019153618204 XXZXIZZIZ
-0.019153618204 XYZYIZZIZ
 0.019153618204 XXIXZZIII
 0.019153618204 XYIYZZIII
 0.001516492369 IYZZZYIII
-0.002126287155 ZYZZZYIII
 0.001516492369 IXZZZXIII
-0.002126287155 ZXZZZXIII
 0.001874590600 YXXZZYIII
-0.001874590600 YYXZZXIII
-0.001874590

## 7.2 generate binary matrix G(Gx|Gz) and parity check matrix E

In [142]:
G_x_LiH, G_z_LiH = create_binary_matrix_G(Hami_LiH)
print('G_x: \n', G_x_LiH)
print('G_z: \n', G_z_LiH)

E_LiH = create_parity_check_matrix_E(G_x_LiH, G_z_LiH)
print('E: \n', E_LiH)

G_x: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
G_z: 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]]
E: 
 [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 7.3 get generators and corresponding paulix_ops using kernel(E)

In [143]:
kernel_vectors_LiH = kernel_of_E(E_LiH)
print('Kernel: \n', kernel_vectors_LiH)

Kernel: 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1]]


In [144]:
generators_LiH = get_generators_from_kernel(kernel_vectors_LiH)
print('Generators: \n', generators_LiH)

rechecked_generators_LiH, paulix_ops_LiH = get_paulix_ops_from_generators(generators_LiH)
print('\nRechecked generators: \n', rechecked_generators_LiH)
print('\npaulix_ops: \n', paulix_ops_LiH)

Generators: 
 1.00000000 IIIIIIZZIIII
1.00000000 IIIIIIIIZZII
1.00000000 ZIZIZIZIZIZI
1.00000000 IZIZIZZIZIIZ

Rechecked generators: 
 1.00000000 IIIIIIZZIIII
1.00000000 IIIIIIIIZZII
1.00000000 ZIZIZIZIZIZI
1.00000000 IZIZIZZIZIIZ

paulix_ops: 
 1.00000000 IIIIIIIXIIII
1.00000000 IIIIIIIIIXII
1.00000000 IIIIIIIIIIXI
1.00000000 IIIIIIIIIIIX


## 7.4 get optimal sector and adjust Hamilton

In [145]:
# optimal sector
n_electrons = 4
optimal_sector_LiH = optimal_sector(Hami_LiH, rechecked_generators_LiH, n_electrons)
print('optimal sector: ', optimal_sector_LiH)

optimal sector:  [1, 1, 1, 1]


In [146]:
Hami_LiH_adjusted = adjust_hamilton_by_optimal_sector(Hami_LiH, paulix_ops_LiH, optimal_sector_LiH)
Hami_LiH_adjusted

-2.46399813 IIIIIIIIIIII
1.22666875 ZIIIIIIIIIII
1.22666875 IZIIIIIIIIII
0.40186073 ZZIIIIIIIIII
0.01975354 YZYIIIIIIIII
0.04472919 YIYIIIIIIIII
0.01975354 XZXIIIIIIIII
0.04472919 XIXIIIIIIIII
0.00151649 YZZZYIIIIIII
-0.00212629 YIZZYIIIIIII
0.00151649 XZZZXIIIIIII
-0.00212629 XIZZXIIIIIII
-0.04229471 YZZZZZZZZZYI
-0.07201792 YIZZZZZZZZYI
-0.04229471 XZZZZZZZZZXI
-0.07201792 XIZZZZZZZZXI
0.01975354 IYZYIIIIIIII
0.04472919 ZYZYIIIIIIII
0.01975354 IXZXIIIIIIII
0.04472919 ZXZXIIIIIIII
0.01107110 YXXYIIIIIIII
-0.01107110 YYXXIIIIIIII
-0.01107110 XXYYIIIIIIII
0.01107110 XYYXIIIIIIII
-0.00187459 YXIXYIIIIIII
-0.00187459 YYIYYIIIIIII
-0.00187459 XXIXXIIIIIII
-0.00187459 XYIYXIIIIIII
0.01915362 YXIXZZZZZZYI
0.01915362 YYIYZZZZZZYI
0.01915362 XXIXZZZZZZXI
0.01915362 XYIYZZZZZZXI
0.00151649 IYZZZYIIIIII
-0.00212629 ZYZZZYIIIIII
0.00151649 IXZZZXIIIIII
-0.00212629 ZXZZZXIIIIII
0.00187459 YXXZZYIIIIII
-0.00187459 YYXZZXIIIIII
-0.00187459 XXYZZYIIIIII
0.00187459 XYYZZXIIIIII
0.00294540 YXIIXYIIIIII

## 7.5 construct unitary U, and get H' = U†HU

In [148]:
Us_LiH = construct_U(rechecked_generators_LiH, paulix_ops_LiH)
Hami_LiH_prime = unitary_transform(Us_LiH, Hami_LiH_adjusted)
print("Hami_LiH_prime: \n", Hami_LiH_prime)

Hami_LiH_prime: 
 -2.46399813+0.00000000j IIIIIIIIIIII
1.22666875+0.00000000j ZIIIIIIIIIII
1.22666875+0.00000000j IZIIIIIIIIII
0.40186073+0.00000000j ZZIIIIIIIIII
0.01975354+0.00000000j YZYIIIIIIIII
0.04472919+0.00000000j YIYIIIIIIIII
0.01975354+0.00000000j XZXIIIIIIIII
0.04472919+0.00000000j XIXIIIIIIIII
0.00151649+0.00000000j YZZZYIIIIIII
-0.00212629+0.00000000j YIZZYIIIIIII
0.00151649+0.00000000j XZZZXIIIIIII
-0.00212629+0.00000000j XIZZXIIIIIII
0.04229471+0.00000000j XZIZIZZXZXII
0.07201792+0.00000000j XIIZIZZXZXII
-0.04229471+0.00000000j XZZZZZIXIXXI
-0.07201792+0.00000000j XIZZZZIXIXXI
0.01975354+0.00000000j IYZYIIIIIIII
0.04472919+0.00000000j ZYZYIIIIIIII
0.01975354+0.00000000j IXZXIIIIIIII
0.04472919+0.00000000j ZXZXIIIIIIII
0.01107110+0.00000000j YXXYIIIIIIII
-0.01107110+0.00000000j YYXXIIIIIIII
-0.01107110+0.00000000j XXYYIIIIIIII
0.01107110+0.00000000j XYYXIIIIIIII
-0.00187459+0.00000000j YXIXYIIIIIII
-0.00187459+0.00000000j YYIYYIIIIIII
-0.00187459+0.00000000j XXIXXIIIIIII


## 7.6 eliminated extra qubits

In [149]:
Hami_LiH_prime_eliminated = eliminate_qubits(Hami_LiH_prime, paulix_ops_LiH)
print("Hami_LiH_prime_eliminated: \n", Hami_LiH_prime_eliminated)

Hami_LiH_prime_eliminated: 
 -2.30752540+0.00000000j IIIIIIII
1.22666875+0.00000000j ZIIIIIII
1.22666875+0.00000000j IZIIIIII
0.40186073+0.00000000j ZZIIIIII
0.01975354+0.00000000j YZYIIIII
0.04472919+0.00000000j YIYIIIII
0.01975354+0.00000000j XZXIIIII
0.04472919+0.00000000j XIXIIIII
0.00151649+0.00000000j YZZZYIII
-0.00212629+0.00000000j YIZZYIII
0.00151649+0.00000000j XZZZXIII
-0.00212629+0.00000000j XIZZXIII
0.04229471+0.00000000j XZIZIZZZ
0.07201792+0.00000000j XIIZIZZZ
-0.04229471+0.00000000j XZZZZZII
-0.07201792+0.00000000j XIZZZZII
0.01975354+0.00000000j IYZYIIII
0.04472919+0.00000000j ZYZYIIII
0.01975354+0.00000000j IXZXIIII
0.04472919+0.00000000j ZXZXIIII
0.01107110+0.00000000j YXXYIIII
-0.01107110+0.00000000j YYXXIIII
-0.01107110+0.00000000j XXYYIIII
0.01107110+0.00000000j XYYXIIII
-0.00187459+0.00000000j YXIXYIII
-0.00187459+0.00000000j YYIYYIII
-0.00187459+0.00000000j XXIXXIII
-0.00187459+0.00000000j XYIYXIII
-0.01915362+0.00000000j XXZXIZZZ
-0.01915362+0.00000000j XYZYIZZ

## 7.7 eigenvalues results

In [150]:
print('Test: Hami_reduced\n')
print("Sorted Eigenvalues: ")
eigvals_reduced = np.linalg.eigvals(Hami_LiH_prime_eliminated.matrix)
# print(np.sort(eigvals_reduced))
print('min eigenvalue: ', min(eigvals_reduced))

Test: Hami_reduced

Sorted Eigenvalues: 
min eigenvalue:  (-6.754943641406253-8.555576742019273e-29j)


In [151]:
print('Test: Hami_reduced\n')
print("Sorted Eigenvalues: ")
eigvals_reduced = np.linalg.eigvals(Hami_LiH_reduced_by_qml.matrix)
# print(np.sort(eigvals_reduced))
print('min eigenvalue: ', min(eigvals_reduced))

Test: Hami_reduced

Sorted Eigenvalues: 
min eigenvalue:  (-6.754943641406352-2.6469779210724035e-23j)
